# Recherche Predictive - Vision 2026 (XGBoost Ultra-Optimise)

Ce notebook demontre la puissance d'un **XGBoost unique** lorsqu'il est alimente par des variables contextuelles riches (jours feries, lags adaptatifs) et une recherche par grille approfondie pour capturer les pics extremes.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import joblib
import os

In [ ]:
# 1. Chargement et Feature Engineering
df_adm = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df_adm['date_entree'] = pd.to_datetime(df_adm['date_entree'])
daily_data = df_adm.groupby('date_entree').size().rename('admissions').reset_index()
daily_data = daily_data.set_index('date_entree').asfreq('D', fill_value=0)

def create_features(df):
    df = df.copy()
    # Temps Cyclique
    df['month_sin'] = np.sin(2 * np.pi * df.index.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df.index.month / 12)
    df['day_sin'] = np.sin(2 * np.pi * df.index.dayofweek / 7)
    df['day_cos'] = np.cos(2 * np.pi * df.index.dayofweek / 7)
    
    # Jours Feries (Impact majeur sur les pics de decembre)
    holidays = ['2024-01-01', '2024-04-01', '2024-05-01', '2024-05-08', 
                '2024-05-09', '2024-05-20', '2024-07-14', '2024-08-15', 
                '2024-11-01', '2024-11-11', '2024-12-25']
    df['is_holiday'] = df.index.strftime('%Y-%m-%d').isin(holidays).astype(int)
    
    df['dayofyear'] = df.index.dayofyear
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    
    # Lags et Fenetres Mobiles
    df['lag1'] = df['admissions'].shift(1)
    df['lag7'] = df['admissions'].shift(7)
    df['lag14'] = df['admissions'].shift(14)
    df['roll_mean_7'] = df['admissions'].shift(1).rolling(window=7).mean()
    df['roll_std_7'] = df['admissions'].shift(1).rolling(window=7).std()
    return df.dropna()

features_df = create_features(daily_data)
FEATURES = ['month_sin', 'month_cos', 'day_sin', 'day_cos', 'is_holiday', 'dayofyear', 
            'weekofyear', 'lag1', 'lag7', 'lag14', 'roll_mean_7', 'roll_std_7']
TARGET = 'admissions'

train = features_df.iloc[:-30]
test = features_df.iloc[-30:]
X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]

## 2. Deep GridSearchCV (XGBoost Only)
Nous cherchons des arbres plus profonds (`max_depth` jusqu'a 10) pour capturer les non-linearites extremes de decembre.

In [ ]:
param_grid = {
    'n_estimators': [1000, 2000],
    'learning_rate': [0.01, 0.05],
    'max_depth': [6, 8, 10],
    'objective': ['count:poisson'] # Meilleur pour les donnees d'admissions
}

tscv = TimeSeriesSplit(n_splits=3)
grid = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=tscv, scoring='neg_mean_absolute_error', verbose=1)
grid.fit(X_train, y_train)

print(f"Best Hyperparams : {grid.best_params_}")
model = grid.best_estimator_

In [ ]:
y_pred = model.predict(X_test)
res = pd.DataFrame({'Reel': y_test, 'Pred': y_pred}, index=test.index)

fig = go.Figure()
fig.add_trace(go.Scatter(x=res.index, y=res['Reel'], name='Reel (Pitie)', line=dict(color='#005ba1', width=3)))
fig.add_trace(go.Scatter(x=res.index, y=res['Pred'], name='XGBoost (Ultra-Tuned)', line=dict(color='#c8102e', dash='dot', width=3)))
fig.update_layout(title='Precision au Sommet : Capture des Pics de Decembre', template='plotly_dark')
fig.show()

print(f"MAE de Decembre : {mean_absolute_error(y_test, y_pred):.2f}")